# magic commands

In [ ]:
#!lsmagic

In [ ]:
#!about

In [ ]:
var x = 10;
var q = 0;

In [ ]:
#!who

In [ ]:
#!whos

# display

In [ ]:
Console.WriteLine("Hello world");

In [ ]:
Console.WriteLine(h2("hello world"));

In [ ]:
display("hello world");

In [ ]:
display(h2("hello world"))

In [ ]:
h2[style: "color: red"]("hello world")

In [ ]:
h2[style: "color: red"]("hello world")

In [ ]:
var pocketViewTagMethods = typeof(PocketViewTags).GetProperties().Select(m => m.Name);
    
string.Join(",", pocketViewTagMethods)

In [ ]:
select(option("first"), option("second"))

# #load

In [ ]:
#load "C:\Users\dcost\source\repos\JupyterNotebooks\start.csx"

In [ ]:
Add(100, 200)

# object formatter

In [ ]:
class Student { public string FirstName { get; set; } public string LastName { get; set; }}

In [ ]:
var student = new Student { FirstName = "John", LastName = "Smith" };
display(student)

In [ ]:
Formatter.Register<Student>((student, writer) =>
{
    writer.Write(b[style: "color: green"]($"First name: {student.FirstName}, last name: {student.LastName}"));
}, "text/html");

In [ ]:
display(student)

In [ ]:
//Formatter.ResetToDefault(); // this resets all object formatters (inclusing charts objects!), not recommended

# charts

In [ ]:
using XPlot.Plotly;

In [ ]:
var numbers = Enumerable.Range(1, 100).ToList();

In [ ]:
numbers

In [ ]:
Random rnd = new Random();
var randomNumbersA = numbers.Select(_ => rnd.Next(100));
var randomNumbersB = numbers.Select(_ => rnd.Next(50));

In [ ]:
randomNumbersA.ToList()

#### object formatter for list of ints

In [ ]:
static string BuildHideRowsScript(long uniqueId)
{
    var script = $"var allRows = document.querySelectorAll('#table_{uniqueId} tbody tr:nth-child(n)'); ";
    script += "for (let i = 0; i < allRows.length; i++) { allRows[i].style.display='none'; } ";
    return script;
}

static string BuildPageScript(long uniqueId, int size)
{
    var script = $"var page = parseInt(document.querySelector('#page_{uniqueId}').innerHTML) - 1; ";
    script += $"var pageRows = document.querySelectorAll(`#table_{uniqueId} tbody tr:nth-child(n + ${{page * {size} + 1 }})`); ";
    script += $"for (let j = 0; j < {size}; j++) {{ pageRows[j].style.display='table-row'; }} ";
    return script;
}

static string GotoPageIndex(long uniqueId, long page)
{
    var script = $"document.querySelector('#page_{uniqueId}').innerHTML = {page + 1}; ";
    return script;
}

static string UpdatePageIndex(long uniqueId, int step, long maxPage)
{
    var script = $"var page = parseInt(document.querySelector('#page_{uniqueId}').innerHTML) - 1; ";
    script += $"page = parseInt(page) + parseInt({step}); ";
    script += $"page = page < 0 ? 0 : page; ";
    script += $"page = page > {maxPage} ? {maxPage} : page; ";
    script += $"document.querySelector('#page_{uniqueId}').innerHTML = page + 1; ";
    return script;
}

In [ ]:
using Microsoft.AspNetCore.Html;

Formatter.Register<List<int>>((ints, writer) =>
            {
                const int MAX = 10000;
                const int SIZE = 10;

                var uniqueId = DateTime.Now.Ticks;

                if (ints.Count > SIZE)
                {
                    var maxMessage = ints.Count > MAX ? $" (showing a max of {MAX} rows)" : string.Empty;
                    var title = h3[style: "text-align: center;"]($"List of ints - {ints.Count} rows {maxMessage}");

                    //var header = new List<IHtmlContent>
                    //{
                    //    th(i("index"))
                    //};
                    //header.AddRange(df.Columns.Select(c => (IHtmlContent)th(c.Name)));

                    // table body
                    var maxRows = Math.Min(MAX, ints.Count);
                    var rows = new List<List<IHtmlContent>>();
                    for (var index = 0; index < Math.Min(MAX, ints.Count); index++)
                    {
                        var cells = new List<IHtmlContent>
                        {
                            td(i((index)))
                        };
                        //foreach (var obj in ints[index])
                        //{
                        //    cells.Add(td(obj));
                        //}
                        cells.Add(td(ints[index]));
                        rows.Add(cells);
                    }

                    //navigator      
                    var footer = new List<IHtmlContent>();
                    BuildHideRowsScript(uniqueId);

                    if (ints.Count > SIZE)
                    {
                        var paginateScriptFirst = BuildHideRowsScript(uniqueId) + GotoPageIndex(uniqueId, 0) + BuildPageScript(uniqueId, SIZE);
                        footer.Add(button[style: "margin: 2px;", onclick: paginateScriptFirst]("⏮"));

                        var paginateScriptPrevTen = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, -10, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
                        footer.Add(button[style: "margin: 2px;", onclick: paginateScriptPrevTen]("⏪"));

                        var paginateScriptPrev = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, -1, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
                        footer.Add(button[style: "margin: 2px;", onclick: paginateScriptPrev]("◀️"));

                        footer.Add(b[style: "margin: 2px;"]("Page"));
                        footer.Add(b[id: $"page_{uniqueId}", style: "margin: 2px;"]("1"));

                        var paginateScriptNext = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, 1, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
                        footer.Add(button[style: "margin: 2px;", onclick: paginateScriptNext]("▶️"));

                        var paginateScriptNextTen = BuildHideRowsScript(uniqueId) + UpdatePageIndex(uniqueId, 10, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
                        footer.Add(button[style: "margin: 2px;", onclick: paginateScriptNextTen]("⏩"));

                        var paginateScriptLast = BuildHideRowsScript(uniqueId) + GotoPageIndex(uniqueId, (maxRows - 1) / SIZE) + BuildPageScript(uniqueId, SIZE);
                        footer.Add(button[style: "margin: 2px;", onclick: paginateScriptLast]("⏭️"));
                    }
                    else
                    {
                        BuildHideRowsScript(uniqueId);
                        footer.Add(b[style: "margin: 2px;"]("Page"));
                        footer.Add(b[id: $"page_{uniqueId}", style: "margin: 2px;"]("0"));
                    }

                    //table
                    var t = table[id: $"table_{uniqueId}"](
                        caption(title),
                        thead(tr(header)),
                        tbody(rows.Select(r => tr[style: "display: none"](r))),
                        tfoot(tr(td[style: "text-align: center;"](footer)))
                    );
                    writer.Write(t);

                }
                else
                {
                    var rows = new List<List<IHtmlContent>>();
                    for (var index = 0; index < ints.Count; index++)
                    {
                        var cells = new List<IHtmlContent>
                        {
                            td(i((index)))
                        };
                        //foreach (var obj in ints[index])
                        //{
                            //cells.Add(td(obj));
                        //}
                        cells.Add(td(ints[index]));
                        rows.Add(cells);
                    }

                    //table
                    var t = table[id: $"table_{uniqueId}"](
                        thead(tr(header)),
                        tbody(rows.Select(r => tr(r)))
                    );
                    writer.Write(t);
                }

                //show first page
                writer.Write($"<script>{BuildPageScript(uniqueId, SIZE)}</script>");

            }, "text/html");

In [ ]:
randomNumbersA.ToList()

#### box plot segmentation

In [ ]:
var segmentationDiagram = Chart.Plot(new[] {
    new Graph.Box { y = randomNumbersA, name = "First set of numbers" },
    new Graph.Box { y = randomNumbersB, name = "Second set of numbers" }
});

var layout = new Layout.Layout()
{
    title = "Box plot segmentation"
};
segmentationDiagram.WithLayout(layout);

display(segmentationDiagram);